# Construção do pipeline

Nesta seção será construido o pipilene para o modelo transformações escolhidas. Este notebook será usado como referência para a construção de um pacote em python capaz de treinar e preiver com modelo desenvolvido

Sequência para a construção do pipeline

- Divisão nos dados: deve ser selecionado somente as colunas utilizadas para o treinamento do modelo. Os dados também devem ser dividos entre dados de treinamento e de teste.
- Definição das classes para as transformações nos dados
- Definição do  pipeline e treinamento do modelo
- Avaliação do modelo


In [11]:
import pandas as pd
import numpy as np

# tranformação
from sklearn.preprocessing import PowerTransformer

# metodos sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics

# modelos
from sklearn.ensemble import RandomForestClassifier

# para salvar o modelo
import joblib

In [12]:
#leitura dos dados
df = pd.read_csv(".//dados//dados_originais//dados_treinamento.csv" ,sep=',');

# visualiza o dado
df.head()

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,24.0,M,167.3,56.10,15.6,76.0,130.0,44.6,23.1,44.0,208.0,C
1,54.0,F,161.0,51.86,29.9,88.0,154.0,22.1,23.0,36.0,148.0,A
2,34.0,M,171.7,74.22,22.6,86.0,138.0,56.1,11.4,38.0,229.0,C
3,21.0,M,174.0,69.80,10.8,75.0,133.0,48.5,28.1,67.0,248.0,A
4,37.0,F,162.8,53.00,18.4,74.0,116.0,28.5,22.8,51.0,193.0,B


## Divisões dos dados

In [13]:
# seleciona variaveis de treinamento e alvo
var_train=[var for var in df.columns if var != 'class'] 

X = df[var_train]   
y = df['class']    

In [14]:
#Divide os dados
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.1,
                                                    random_state=0)

X_train.shape, X_test.shape

((10847, 11), (1206, 11))

## Classes para transformação dos dados

In [15]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin


class Mapeamento(BaseEstimator, TransformerMixin):    

    def __init__(self, variavel, mapa):        
        '''
        variavel: nome da coluna que será alterada
        mapa: mapa com as alterações
        '''
        self.variavel = variavel
        self.mapa = mapa 

        
    def fit(self, X, y=None):        
        '''
        X: dataframe com os dados de treinamento        
        y: serie com o rótulos do dados de treinamento
        '''        
        return self
    

    def transform(self, X):
        '''
        X: dataframe com os dados de treinamento        
        '''        
        X = X.copy()        
        X[self.variavel] = X[self.variavel].map(self.mapa)
        return X
    
    
    
    
    
class transf_yeo_johnson(BaseEstimator, TransformerMixin):    

    def __init__(self, colunas):
        '''
        colunas: lista com as colunas que serão transformadas
        '''
        self.colunas = colunas
        self.obj_transf= PowerTransformer(method="yeo-johnson")
    
    
    def fit(self, X, y=None):
        '''
        X: dataframe com os dados de treinamento        
        y: serie com o rótulos do dados de treinamento
        '''        
        self.obj_transf= self.obj_transf.fit(X[self.colunas].values) 
        return self  
    

    def transform(self, X):
        '''
        X: dataframe com os dados de treinamento        
        '''
        X = X.copy()
        X_array = X[self.colunas].values
        X[self.colunas] = self.obj_transf.transform(X_array)
        return X


## Configurações

In [16]:
# Definições gerais
gender_map = {'M': 0, 'F': 1}

CONTINUOUS_VAR=['age',
                'height_cm',
                'weight_kg',
                'body fat_%', 
                'diastolic',                
                'systolic',
                'gripForce',
                'sit and bend forward_cm',
                'sit-ups counts',
                'broad jump_cm']


## Pipeline

In [17]:
# Construção do pipeline
model_pipe= Pipeline([     
    ('map_gender', Mapeamento(variavel='gender',mapa=gender_map)),
    ('YeoYohnson_trans', transf_yeo_johnson(CONTINUOUS_VAR)),           
    ('randForest', RandomForestClassifier(max_depth=14,
                             max_features=8,
                             n_estimators=80,
                             bootstrap=True,
                             max_samples=None,
                             random_state=0))
    ])

In [18]:
# Fita o pipeline
model_pipe.fit(X_train,y_train)

Pipeline(steps=[('map_gender',
                 Mapeamento(mapa={'F': 1, 'M': 0}, variavel='gender')),
                ('YeoYohnson_trans',
                 transf_yeo_johnson(colunas=['age', 'height_cm', 'weight_kg',
                                             'body fat_%', 'diastolic',
                                             'systolic', 'gripForce',
                                             'sit and bend forward_cm',
                                             'sit-ups counts',
                                             'broad jump_cm'])),
                ('randForest',
                 RandomForestClassifier(max_depth=14, max_features=8,
                                        n_estimators=80, random_state=0))])

## Avaliação do modelo

In [19]:
# modelos previstos
y_hat = model_pipe.predict(X_test)

print('Valores previstos: ', y_hat[0:10])
print('Valores reais: ', y_test.iloc[0:10].to_list())
print("Acuracia:", metrics.accuracy_score(y_test, y_hat))

Valores previstos:  ['B' 'C' 'A' 'B' 'A' 'A' 'B' 'D' 'B' 'B']
Valores reais:  ['C', 'C', 'A', 'C', 'B', 'A', 'B', 'D', 'B', 'A']
Acuracia: 0.7271973466003316


### Salva o pipeline

In [20]:
# salva o pipeline
joblib.dump(model_pipe, './/modelo_treinado//model_pipe.joblib')

['.//modelo_treinado//model_pipe.joblib']